In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ruptures as rpt
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
import lightgbm as lgb
import keras_tuner as kt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, BatchNormalization, Activation, Dropout, Flatten, Dense, Add
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.layers import (
    Input, Embedding, Flatten, Conv1D, BatchNormalization,
    Activation, Dropout, Add, Dense, Concatenate, GRU
)
import shap


In [ ]:
hate_crime_path = "hate_crime.csv"
data_crime = pd.read_csv(hate_crime_path)


In [ ]:
data_crime.head(5)

In [ ]:
data_crime.columns

In [ ]:
data_crime.describe()

In [ ]:
data_crime.info()

In [ ]:

# Converting incident_date to datetime
data_crime['incident_date'] = pd.to_datetime(data_crime['incident_date'], errors='coerce')

# Adding year column for trend analysis
data_crime['year'] = data_crime['incident_date'].dt.year

# Top 10 hate crime bias types
plt.figure(figsize=(10,5))
data_crime['bias_desc'].value_counts().head(10).plot(kind='barh')
plt.title('Top 10 Hate Crime Bias Types')
plt.xlabel('Number of Incidents')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()




In [ ]:
# Top 10 offense types
plt.figure(figsize=(10,9))
data_crime['offense_name'].value_counts().head(10).plot(kind='bar')
plt.title('Top 10 Offense Types')
plt.ylabel('Number of Incidents')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()



In [ ]:
# Top 15 states by number of incidents
plt.figure(figsize=(12,6))
data_crime['state_name'].value_counts().head(15).plot(kind='bar')
plt.title('Top 15 States by Hate Crime Count')
plt.ylabel('Number of Incidents')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
# 3. Initial Cleaning of data
data_crime.drop("pub_agency_unit", axis=1, inplace=True)
data_crime['incident_date'] = pd.to_datetime(data_crime['incident_date'], errors='coerce')
data_crime['year'] = data_crime['incident_date'].dt.year


In [ ]:
# 4. missing numerical fields
fill_zero_cols = [
    'adult_victim_count', 'juvenile_victim_count', 'adult_offender_count',
    'juvenile_offender_count', 'total_individual_victims'
]
data_crime[fill_zero_cols] = data_crime[fill_zero_cols].fillna(0)


In [ ]:
# 5. Categorize Bias
def categorize_bias(bias):
    if isinstance(bias, str):
        if 'Race' in bias or 'Black' in bias or 'White' in bias or 'Asian' in bias:
            return 'Race'
        elif 'Jewish' in bias or 'Islamic' in bias or 'Christian' in bias or 'Religion' in bias:
            return 'Religion'
        elif 'Gender' in bias or 'Sexual' in bias or 'Transgender' in bias:
            return 'Gender/Sexuality'
        elif 'Disability' in bias:
            return 'Disability'
        elif 'Ethnicity' in bias or 'Hispanic' in bias:
            return 'Ethnicity'
    return 'Other'

data_crime['bias_category'] = data_crime['bias_desc'].apply(categorize_bias)

# 6. Group by Year and Bias
yearly_bias = data_crime.groupby(['year', 'bias_category']).agg({
    'victim_count': 'sum',
    'total_offender_count': 'sum',
    'offense_name': 'count'
}).rename(columns={'offense_name': 'incident_count'}).reset_index()


In [ ]:
# 7. Feature Engineering
yearly_bias['incident_rolling'] = yearly_bias.groupby('bias_category')['incident_count'].transform(lambda x: x.rolling(window=3, min_periods=1).mean())
yearly_bias['yoy_change'] = yearly_bias.groupby('bias_category')['incident_count'].pct_change()
yearly_bias['incident_lag1'] = yearly_bias.groupby('bias_category')['incident_count'].shift(1)
yearly_bias['incident_lag2'] = yearly_bias.groupby('bias_category')['incident_count'].shift(2)
model_data = yearly_bias.dropna()

In [ ]:
# 8. One-Hot Encode Bias Category
encoder = OneHotEncoder(sparse_output=False, drop='first')
bias_encoded = encoder.fit_transform(model_data[['bias_category']])
bias_encoded_df = pd.DataFrame(bias_encoded, columns=encoder.get_feature_names_out(['bias_category']), index=model_data.index)

# 9. Build Feature Matrix
X = pd.concat([
    model_data[['year', 'incident_rolling', 'yoy_change', 'incident_lag1', 'incident_lag2']],
    bias_encoded_df
], axis=1)
y = model_data['incident_count']

In [ ]:
# 10. Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# 11. Train & Tune Random Forest
rf_param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}
rf = RandomForestRegressor(random_state=42)
grid_search_rf = GridSearchCV(estimator=rf, param_grid=rf_param_grid, cv=3, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search_rf.fit(X_train, y_train)
best_rf = grid_search_rf.best_estimator_
y_pred_rf_best = best_rf.predict(X_test)
rmse_best_rf = mean_squared_error(y_test, y_pred_rf_best, squared=False)
mae_best_rf = mean_absolute_error(y_test, y_pred_rf_best)


In [ ]:
print("Random Forest Best Params:", grid_search_rf.best_params_)
print(f"Random Forest RMSE: {rmse_best_rf:.2f}, MAE: {mae_best_rf:.2f}")

In [ ]:
# 12. Train & Tune LightGBM
lgb_param_grid = {
    'n_estimators': [100, 200],
    'learning_rate': [0.05, 0.1],
    'max_depth': [3, 5, 10],
    'num_leaves': [15, 31, 63],
    'min_child_samples': [5, 10]
}
lgb_model = lgb.LGBMRegressor(random_state=42)
grid_search_lgb = GridSearchCV(estimator=lgb_model, param_grid=lgb_param_grid, cv=3, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search_lgb.fit(X_train, y_train)
best_lgb = grid_search_lgb.best_estimator_
y_pred_lgb = best_lgb.predict(X_test)
rmse_lgb = mean_squared_error(y_test, y_pred_lgb, squared=False)
mae_lgb = mean_absolute_error(y_test, y_pred_lgb)


In [ ]:
print("LightGBM Best Params:", grid_search_lgb.best_params_)
print(f"LightGBM RMSE: {rmse_lgb:.2f}, MAE: {mae_lgb:.2f}")

TCN

In [ ]:
# STEP 2: Define columns based on your schema
categorical_cols = ['state_abbr', 'offender_race', 'offender_ethnicity', 'offense_name', 'bias_desc']
numerical_cols = [
    'adult_victim_count', 'juvenile_victim_count', 'total_offender_count',
    'adult_offender_count', 'juvenile_offender_count',
    'victim_count', 'total_individual_victims'
]
target_col = 'victim_count' 


In [ ]:

# STEP 3: Encode categorical features and scale numerics
data_crime = data_crime.dropna(subset=numerical_cols + categorical_cols + [target_col])  # Drop rows with missing critical data

label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    data_crime[col + '_enc'] = le.fit_transform(data_crime[col])
    label_encoders[col] = le

scaler = StandardScaler()
data_crime[numerical_cols] = scaler.fit_transform(data_crime[numerical_cols])


In [ ]:
# STEP 4: Build sequences
WINDOW_SIZE = 5
cat_feature_array = data_crime[[col + '_enc' for col in categorical_cols]].values
num_feature_array = data_crime[numerical_cols].values
y_values = data_crime[target_col].values

X_seq_num, X_seq_cat, y_seq = [], [], []
for i in range(WINDOW_SIZE, len(data_crime)):
    X_seq_num.append(num_feature_array[i - WINDOW_SIZE:i])
    X_seq_cat.append(cat_feature_array[i])
    y_seq.append(y_values[i])

X_seq_num = np.array(X_seq_num)
X_seq_cat = np.array(X_seq_cat)
y_seq = np.array(y_seq)


In [ ]:

# STEP 5: Train/test split
train_size = int(len(X_seq_num) * 0.8)
X_train_seq = X_seq_num[:train_size]
X_test_seq = X_seq_num[train_size:]
X_train_cat = X_seq_cat[:train_size]
X_test_cat = X_seq_cat[train_size:]
y_train = y_seq[:train_size]
y_test = y_seq[train_size:]



In [ ]:
# STEP 6: Build TCN model with embeddings
def build_tcn_model(hp):
    seq_input = Input(shape=(X_train_seq.shape[1], X_train_seq.shape[2]), name="seq_input")

    embed_inputs = []
    embeddings = []
    for i, col in enumerate(categorical_cols):
        num_classes = len(label_encoders[col].classes_)
        cat_input = Input(shape=(1,), name=f'{col}_input')
        embed = Embedding(input_dim=num_classes, output_dim=hp.Int(f'{col}_embed_dim', 4, 16))(cat_input)
        embed = Flatten()(embed)
        embed_inputs.append(cat_input)
        embeddings.append(embed)

    # TCN layers
    x = seq_input
    for i in range(hp.Int('num_blocks', 1, 2)):
        res = x
        x = Conv1D(
            filters=hp.Int(f'filters_{i}', 32, 128, step=32),
            kernel_size=hp.Choice(f'kernel_size_{i}', [2, 3, 4]),
            padding='causal'
        )(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = Dropout(hp.Float(f'dropout_{i}', 0.1, 0.5, step=0.1))(x)
        if res.shape[-1] == x.shape[-1]:
            x = Add()([x, res])

    x = Flatten()(x)
    merged = Concatenate()([x] + embeddings)
    x = Dense(hp.Int('dense_units', 32, 128, step=32), activation='relu')(merged)
    x = Dropout(hp.Float('dense_dropout', 0.1, 0.5, step=0.1))(x)
    output = Dense(1)(x)

    model = Model(inputs=[seq_input] + embed_inputs, outputs=output)
    model.compile(
        optimizer=Adam(learning_rate=hp.Choice('lr', [1e-2, 1e-3, 5e-4])),
        loss='mse',
        metrics=['mae']
    )
    return model

In [ ]:
# STEP 7: Hyperparameter tuning
tuner = kt.RandomSearch(
    build_tcn_model,
    objective='val_mae',
    max_trials=10,
    directory='tcn_tuning_final',
    project_name='hatecrime_tcn_embed'
)

tuner.search_space_summary()

early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

tuner.search(
    x=[X_train_seq] + [X_train_cat[:, i] for i in range(X_train_cat.shape[1])],
    y=y_train,
    validation_split=0.2,
    epochs=50,
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)


In [ ]:
# STEP 8: Evaluate best model
best_tcn_model = tuner.get_best_models(1)[0]
preds = best_tcn_model.predict([X_test_seq] + [X_test_cat[:, i] for i in range(X_test_cat.shape[1])]).flatten()
rmse = mean_squared_error(y_test, preds, squared=False)
mae = mean_absolute_error(y_test, preds)

print(f"\n Tuned TCN RMSE: {rmse:.2f}")
print(f" Tuned TCN MAE: {mae:.2f}")

GRU Model

In [ ]:
# Train/test split
train_size = int(len(X_seq_num) * 0.8)
X_train_seq, X_test_seq = X_seq_num[:train_size], X_seq_num[train_size:]
X_train_cat, X_test_cat = X_seq_cat[:train_size], X_seq_cat[train_size:]
y_train_seq, y_test_seq = y_seq[:train_size], y_seq[train_size:]


In [ ]:
# STEP 4: Build ST-GRU Model with Embeddings
def build_stgru_model(hp):
    # Numerical sequence input
    seq_input = Input(shape=(WINDOW_SIZE, X_train_seq.shape[2]), name="seq_input")

    # Categorical embeddings
    embed_inputs = []
    embeddings = []
    for i, col in enumerate(categorical_cols):
        vocab_size = len(label_encoders[col].classes_)
        input_cat = Input(shape=(1,), name=f"{col}_input")
        embed_dim = hp.Int(f"{col}_embed_dim", 4, 16)
        embed = Embedding(input_dim=vocab_size, output_dim=embed_dim)(input_cat)
        embed = Flatten()(embed)
        embed_inputs.append(input_cat)
        embeddings.append(embed)

    # GRU output from sequence
    gru_units = hp.Int("gru_units", 32, 128, step=32)
    x = GRU(gru_units)(seq_input)

    # Merge all
    x = Concatenate()([x] + embeddings)
    dense_units = hp.Int("dense_units", 32, 128, step=32)
    x = Dense(dense_units, activation='relu')(x)
    x = Dropout(hp.Float("dropout", 0.1, 0.5, step=0.1))(x)
    output = Dense(1)(x)

    model = Model(inputs=[seq_input] + embed_inputs, outputs=output)
    model.compile(
        optimizer=Adam(hp.Choice("lr", [1e-2, 1e-3, 5e-4])),
        loss='mse',
        metrics=['mae']
    )
    return model


In [ ]:
# STEP 5: Hyperparameter Tuning with KerasTuner
tuner = kt.RandomSearch(
    build_stgru_model,
    objective='val_mae',
    max_trials=2,
    executions_per_trial=1,
    directory='stgru_tuning_final',
    project_name='hatecrime_stgru_embed'
)

tuner.search_space_summary()

tuner.search(
    x=[X_train_seq] + [X_train_cat[:, i] for i in range(X_train_cat.shape[1])],
    y=y_train_seq,
    validation_split=0.2,
    epochs=50,
    batch_size=16,
    verbose=1
)


In [ ]:
# STEP 6: Evaluate the Best Model
best_model = tuner.get_best_models(1)[0]
preds = best_model.predict([X_test_seq] + [X_test_cat[:, i] for i in range(X_test_cat.shape[1])]).flatten()
rmse = mean_squared_error(y_test_seq, preds, squared=False)
mae = mean_absolute_error(y_test_seq, preds)

print(f"\n Tuned ST-GRU RMSE: {rmse:.2f}")
print(f" Tuned ST-GRU MAE: {mae:.2f}")

In [ ]:

# Create SHAP Explainer
explainer = shap.Explainer(best_lgb)

# Use the same test set as in model evaluation
X_test_df = X_test.copy()

# Compute SHAP values
shap_values = explainer(X_test_df)

# Summary Plot (Global Feature Importance)
shap.plots.beeswarm(shap_values)


In [ ]:

shap.plots.bar(shap_values, max_display=10)

shap.plots.scatter(shap_values[:, "incident_lag1"], color=shap_values)

sample_index = 10
shap.plots.waterfall(shap_values[sample_index])


In [ ]:
# Extract city and state
data_crime['city'] = data_crime['pug_agency_name'].str.extract(r'([A-Za-z\s]+)')
data_crime['state'] = data_crime['state_name']

# Simulated sample US cities dataset (replace with full file for production use)
cities_sample =  pd.read_csv("uscities.csv")
# Normalize text
data_crime['city'] = data_crime['city'].str.lower().str.strip()
data_crime['state'] = data_crime['state'].str.lower().str.strip()
cities_sample['city'] = cities_sample['city'].str.lower().str.strip()
cities_sample['state_name'] = cities_sample['state_name'].str.lower().str.strip()

# Merge datasets
merged = data_crime.merge(cities_sample, left_on=['city', 'state'], right_on=['city', 'state_name'], how='left')
merged = merged.rename(columns={'lat': 'latitude', 'lng': 'longitude'})



In [ ]:
import folium

# Filter rows with valid coordinates
map_data = merged.dropna(subset=['latitude', 'longitude'])

# Create a folium map centered on the U.S.
map_center = [39.8283, -98.5795]  # Geographic center of contiguous USA
m = folium.Map(location=map_center, zoom_start=4)

# Add markers to the map (sample if too large)
for _, row in map_data.sample(n=175000, random_state=42).iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=3,
        color='blue',
        fill=True,
        fill_opacity=0.6
    ).add_to(m)

m


In [ ]:
import ruptures as rpt
import matplotlib.pyplot as plt

# Define list of bias categories to analyze
bias_categories = yearly_bias['bias_category'].unique()

# Detect and visualize change points for each category
for category in bias_categories:
    subset = yearly_bias[yearly_bias['bias_category'] == category].sort_values('year')
    signal = subset['incident_count'].values
    years = subset['year'].values

    if len(signal) < 6:
        continue  # skip too-short series

    # Apply PELT change point detection
    model = rpt.Pelt(model="l2").fit(signal)
    change_points = model.predict(pen=10)

    # Plot results
    plt.figure(figsize=(10, 4))
    plt.plot(years, signal, label=f'{category} Incidents')
    for cp in change_points[:-1]:
        plt.axvline(x=years[cp], color='red', linestyle='--', alpha=0.7)
    plt.title(f'Change Point Detection - {category}')
    plt.xlabel('Year')
    plt.ylabel('Incident Count')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    # Optional: Print change years
    change_years = years[change_points[:-1]]
    print(f'{category} Change Years:', change_years.tolist())


In [ ]:
future_df = model_data.copy()
last_known_year = future_df['year'].max()
future_years = [2024, 2025, 2026]

# Columns used for prediction
feature_cols = list(X.columns)

# Storage for future predictions
future_preds = []

# Recursive Forecast Loop
for future_year in future_years:
    new_rows = []

    for bias_cat in future_df['bias_category'].unique():
        # Get last 5 rows for this bias category
        group = future_df[future_df['bias_category'] == bias_cat].sort_values('year').tail(5).copy()

        if group.shape[0] < 2:
            continue  # skip if not enough history

        # Compute new features
        incident_lag1 = group.iloc[-1]['incident_count']
        incident_lag2 = group.iloc[-2]['incident_count']
        rolling_avg = group['incident_count'].rolling(3, min_periods=1).mean().iloc[-1]
        pct_change = (group.iloc[-1]['incident_count'] - group.iloc[-2]['incident_count']) / (group.iloc[-2]['incident_count'] + 1e-6)

        # Construct input row
        input_row = {
            'year': future_year,
            'incident_rolling': rolling_avg,
            'yoy_change': pct_change,
            'incident_lag1': incident_lag1,
            'incident_lag2': incident_lag2
        }

        # Add encoded bias_category columns (same as training one-hot)
        for col in encoder.get_feature_names_out(['bias_category']):
            input_row[col] = 1 if col == f"bias_category_{bias_cat}" else 0

        # Ensure all expected features are present
        input_df = pd.DataFrame([input_row], columns=feature_cols)

        # Predict with LightGBM
        pred = best_lgb.predict(input_df)[0]
        input_row['incident_count'] = pred
        input_row['bias_category'] = bias_cat

        new_rows.append(input_row)

    # Add predictions to future_df so they can feed next year's forecast
    new_df = pd.DataFrame(new_rows)
    future_df = pd.concat([future_df, new_df], ignore_index=True)
    future_preds.append(new_df)

# Combine all forecasts into one DataFrame
future_forecast_df = pd.concat(future_preds, ignore_index=True)


In [ ]:
plt.figure(figsize=(12, 6))
sns.lineplot(data=future_df[future_df['year'] >= 2018], x='year', y='incident_count', hue='bias_category')
plt.title("Forecasted Hate Crime Incidents (2024–2026)")
plt.ylabel("Predicted Incidents")
plt.xlabel("Year")
plt.grid(True)
plt.tight_layout()
plt.show()
